In [4]:
import os
import urllib.request
import datetime

import json
from dotenv import load_dotenv

load_dotenv()

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", os.getenv('NAVER_CLIENT_ID'))
    req.add_header("X-Naver-Client-Secret", os.getenv('NAVER_CLIENT_SECRET'))

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responsedecode = getRequestUrl(url) #[CODE 1]

    if responsedecode == None:
        return None
    else:
        return json.loads(responsedecode)

def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pdate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pdate = pdate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,'org_link':org_link, 'link': link, 'pdate':pdate})
    return

def main():
    node = 'news' #크롤링할 대상
    srctext = input('검색어를 입력하세요: ')
    cnt = 0
    jsonresult = []

    jsonresponse = getNaverSearch(node, srctext, 1, 100) #[CODE 2]
    total = jsonresponse['total']

    while ((jsonresponse != None) and (jsonresponse['display'] != 0)):
        for post in jsonresponse['items']:
            cnt += 1
            getPostData(post, jsonresult, cnt) #[CODE 3]

        start = jsonresponse['start'] + jsonresponse['display']
        jsonresponse = getNaverSearch(node, srctext, start, 100) #[CODE 2]

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srctext, node), 'w', encoding='utf8') as outfile:
        jsonfile = json.dumps(jsonresult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonfile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srctext, node))


if __name__ == '__main__':
    main()

[2025-07-07 14:38:32.481415] Url Request Success
[2025-07-07 14:38:32.611220] Url Request Success
[2025-07-07 14:38:32.752202] Url Request Success
[2025-07-07 14:38:32.908682] Url Request Success
[2025-07-07 14:38:33.072626] Url Request Success
[2025-07-07 14:38:33.242399] Url Request Success
[2025-07-07 14:38:33.415141] Url Request Success
[2025-07-07 14:38:33.591066] Url Request Success
[2025-07-07 14:38:33.796354] Url Request Success
[2025-07-07 14:38:33.973750] Url Request Success
HTTP Error 400: Bad Request
[2025-07-07 14:38:34.044617] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EC%82%AC%EA%B4%80%ED%95%99%EA%B5%90&start=1001&display=100
전체 검색 : 11980 건
가져온 데이터 : 1000 건
인공지능사관학교_naver_news.json SAVED
